In [18]:
#ALL ARCHITECTURES VISUAL

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import os

# ---------------------------------------------------------------------------------------------------
# Load the dataset

path = "/Volumes/dax-hd/project-data/search-files/merged-data.csv"
save_folder = "/Volumes/dax-hd/project-data/images/all_arch_pca"

plot_path = os.path.join('all_arch_pca')

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

df = pd.read_csv(path)

# ---------------------------------------------------------------------------------------------------

selected_columns = [
    "hydrophobic_fitness",
    "isoelectric_point",
    "charge",
    "mass",
    "num_residues",
    "packing_density",
    "budeff_total",
    "evoef2_ref_total", 
    "dfire2_total",
    "rosetta_total",
    "aggrescan3d_avg_value"
]

# ---------------------------------------------------------------------------------------------------

df_selected = df[selected_columns]

df_selected_cleaned = df_selected.dropna()

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected_cleaned)

pca = PCA(n_components=0.95)
principal_components = pca.fit_transform(scaled_features)

pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i}' for i in range(1, pca.n_components_ + 1)])

pca_df.to_csv('pca_results.csv', index=False)

pca_df['architecture_name'] = df['architecture_name'][df_selected_cleaned.index].reset_index(drop=True)

print("Variance explained by each component:")
print(pca.explained_variance_ratio_)

# ---------------------------------------------------------------------------------------------------


plt.figure(figsize=(10, 7))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--')
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.tight_layout()
cumulative_variance_plot_path = os.path.join(save_folder, 'cumulative_explained_variance.png')
plt.savefig(cumulative_variance_plot_path)
plt.close()

# ---------------------------------------------------------------------------------------------------


plt.figure(figsize=(30, 20))
sns.scatterplot(x=pca_df['PC1'], y=pca_df['PC2'], hue=pca_df['architecture_name'], palette='Spectral', legend='full')
plt.title('PCA on Dataset by Protein Architecture')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Architecture', bbox_to_anchor=(1.05, 1), loc='upper left')
scatter_plot_path = os.path.join(save_folder, 'pca_scatter_plot_by_architecture.png')
plt.savefig(scatter_plot_path)
plt.close()

# ---------------------------------------------------------------------------------------------------


for i in range(num_components_to_visualize):
    plt.figure(figsize=(10, 6))
    component_loadings = pca.components_[i]
    indices = np.argsort(abs(component_loadings))[::-1]
    plt.bar(feature_names[indices], component_loadings[indices])
    plt.xticks(rotation=90)
    plt.title(f'PCA Component {i+1} Loadings')
    plt.ylabel('Loading Value')
    feature_contribution_path = os.path.join(save_folder, f'pca_component_{i+1}_loadings.png')
    plt.tight_layout()
    plt.savefig(feature_contribution_path)
    plt.close()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [143]:
#ALL TOPOLOGIES BY ARCHITECTURES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
import seaborn as sns
import os
import json
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

# ---------------------------------------------------------------------------------------------------
# Load the dataset

path = "/Volumes/dax-hd/project-data/search-files/merged-data.csv"
base_save_folder = "/Volumes/dax-hd/project-data/images/pca_topology_"
cath_dict_path = "/Volumes/dax-hd/project-data/search-files/cath-archetype-dict.txt"

df = pd.read_csv(path)

with open(cath_dict_path, 'r') as file:
    cath_dict = json.load(file)

# ---------------------------------------------------------------------------------------------------
# Add the architecture name to df

def add_topology_description(df, cath_dict):
    def get_topology_description(row):
        class_num = str(row['Class number'])
        arch_num = str(row['Architecture number'])
        top_num = str(row['Topology number'])
        try:
            description = cath_dict[class_num][arch_num][top_num]['description']
            return description
        except KeyError:
            return "Unknown"
    
    df['topology_description'] = df.apply(get_topology_description, axis=1)
    return df

# ---------------------------------------------------------------------------------------------------

df = add_topology_description(df, cath_dict)

if not os.path.exists(base_save_folder):
    os.makedirs(base_save_folder)

selected_architectures = ["sandwich (2,60)"]

selected_columns = [
    
    # "composition_ALA","composition_CYS","composition_ASP","composition_GLU","composition_PHE","composition_GLY",
    # "composition_HIS","composition_ILE","composition_LYS","composition_LEU","composition_MET","composition_ASN",
    # "composition_PRO","composition_GLN","composition_ARG","composition_SER","composition_THR","composition_VAL",
    # "composition_TRP","composition_UNK","composition_TYR",
    # "ss_prop_alpha_helix","ss_prop_beta_bridge","ss_prop_beta_strand","ss_prop_3_10_helix","ss_prop_pi_helix",
    # "ss_prop_hbonded_turn","ss_prop_bend","ss_prop_loop",
    "hydrophobic_fitness",
    "isoelectric_point",
    "charge",
    "mass",
    # "num_residues",
    "packing_density",
    # "budeff_total",
    # "evoef2_ref_total",
    # "dfire2_total",
    "rosetta_total",
    "aggrescan3d_avg_value"
]

# ---------------------------------------------------------------------------------------------------

def filter_for_archetypes(df, cath_dict):
    archetype_ids = []
    for _, row in df.iterrows():
        class_num = str(row['Class number'])
        arch_num = str(row['Architecture number'])
        top_num = str(row['Topology number'])
        try:
            protein_id = cath_dict[class_num][arch_num][top_num]['protein_id']
            if protein_id[:4] in row['design_name']:
                archetype_ids.append(row['design_name'])
        except KeyError:
            continue
    return df[df['design_name'].isin(archetype_ids)]

df_filtered = filter_for_archetypes(df, cath_dict)

# ---------------------------------------------------------------------------------------------------

df_selected_cleaned = df_filtered[selected_columns].dropna()

scaler = StandardScaler()

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

for architecture_name in df['architecture_name'].unique():
    if architecture_name not in selected_architectures:
        continue
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_filtered = df[df['architecture_name'] == architecture_name]
    df_selected = df_filtered[selected_columns]
    df_selected_cleaned = df_selected.dropna()
    
    if df_selected_cleaned.empty:
        continue

    save_folder = os.path.join(base_save_folder, architecture_name.replace('/', '_'))
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    
    # Standardize and PCA
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_selected_cleaned)
    robust_scaler = RobustScaler()
    scaled_features_robust = robust_scaler.fit_transform(df_selected_cleaned)
    scaled_features = scaler.fit_transform(df_selected_cleaned)
    pca = PCA(n_components=0.95)
    principal_components = pca.fit_transform(scaled_features)
    pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i}' for i in range(1, pca.n_components_ + 1)])
    pca_df['topology_description'] = df_filtered['topology_description'][df_selected_cleaned.index].reset_index(drop=True)
    
    # pca_df.to_csv("/Volumes/dax-hd/project-data/csv.csv", index=False)


    # ---------------------------------------------------------------------------------------------------
    # Plotting

    plt.figure(figsize=(15, 12))
    unique_topologies = pca_df['topology_description'].unique()
    palette = sns.color_palette('tab20c', n_colors=len(unique_topologies))
    topology_color_mapping = {topology: color for topology, color in zip(unique_topologies, palette)}

    scatter_plot = sns.scatterplot(
        x='PC1', y='PC2',
        hue='topology_description',
        palette=topology_color_mapping, 
        data=pca_df, s=100 
    )
    plt.title(f'PCA for {architecture_name}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')

    # Handling legend
    handles, labels = scatter_plot.get_legend_handles_labels()
    plt.legend([],[], frameon=False)
    plt.savefig(os.path.join(save_folder, 'pca_scatter_by_topology.png'), bbox_inches='tight')
    plt.close()

    # Save legend separately
    fig_legend = plt.figure(figsize=(3, 4))
    plt.figlegend(handles, labels, loc='center')
    plt.savefig(os.path.join(save_folder, 'legend.png'), bbox_inches='tight')
    plt.close(fig_legend)

    # Plotting cumulative variance
    plt.figure(figsize=(10, 7))
    plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--')
    plt.title('Cumulative Explained Variance')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.tight_layout()
    cumulative_variance_plot_path = os.path.join(save_folder, 'cumulative_explained_variance.png')
    plt.savefig(cumulative_variance_plot_path)
    plt.close()

    #Component loading plot
    for i in range(pca.n_components_):
        plt.figure(figsize=(10, 6))
        component_loadings = pca.components_[i]
        indices = np.argsort(abs(component_loadings))[::-1]
        feature_names = np.array(selected_columns)[indices]
        plt.bar(range(len(indices)), component_loadings[indices])
        plt.xticks(range(len(indices)), feature_names, rotation=90)
        plt.title(f'PCA Component {i+1} Loadings')
        plt.ylabel('Loading Value')
        feature_contribution_path = os.path.join(save_folder, f'pca_component_{i+1}_loadings.png')
        plt.tight_layout()
        plt.savefig(feature_contribution_path)
        plt.close()

# ---------------------------------------------------------------------------------------------------


In [49]:
#ALL ARCHITECTURES WITH KDE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import os
import json
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

# ---------------------------------------------------------------------------------------------------
# Load the dataset

path = "/Volumes/dax-hd/project-data/search-files/merged-data.csv"
base_save_folder = "/Volumes/dax-hd/project-data/images/pca_architectures_all"
cath_dict_path = "/Volumes/dax-hd/project-data/search-files/cath-archetype-dict.txt"

df = pd.read_csv(path)

with open(cath_dict_path, 'r') as file:
    cath_dict = json.load(file)

if not os.path.exists(base_save_folder):
    os.makedirs(base_save_folder)

# ---------------------------------------------------------------------------------------------------

selected_columns = [
    "composition_ALA","composition_CYS","composition_ASP","composition_GLU","composition_PHE","composition_GLY",
    "composition_HIS","composition_ILE","composition_LYS","composition_LEU","composition_MET","composition_ASN",
    "composition_PRO","composition_GLN","composition_ARG","composition_SER","composition_THR","composition_VAL",
    "composition_TRP","composition_UNK","composition_TYR",
    "ss_prop_alpha_helix","ss_prop_beta_bridge","ss_prop_beta_strand",
    "ss_prop_3_10_helix","ss_prop_pi_helix",
    "ss_prop_hbonded_turn","ss_prop_bend","ss_prop_loop",
    "hydrophobic_fitness",
    "isoelectric_point",
    "charge",
    "mass",
    "num_residues",
    "packing_density",
    "budeff_total",
    "evoef2_ref_total",
    "dfire2_total",
    "rosetta_total",
    "aggrescan3d_avg_value"
]
# ---------------------------------------------------------------------------------------------------

df_selected = df[selected_columns]

df_selected_cleaned = df_selected.dropna()

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected_cleaned)

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['architecture_name'] = df['architecture_name'][df_selected.index].reset_index(drop=True)

print("Variance explained by each component:")
print(pca.explained_variance_ratio_)


# Plotting PCA scatter with KDE
plt.figure(figsize=(30, 20))
unique_architectures = pca_df['architecture_name'].unique()
palette = sns.color_palette("tab20b") + sns.color_palette("tab20c")
architecture_color_mapping = {architecture: color for architecture, color in zip(unique_architectures, palette)}

# KDE overlay for each architecture
for architecture, group_df in pca_df.groupby('architecture_name'):
    sns.kdeplot(
        x=group_df['PC1'], y=group_df['PC2'], 
        color=architecture_color_mapping[architecture],
        levels=2, fill=True, alpha=0.4
    )

# Scatter plot for visualization
sns.scatterplot(
    x='PC1', y='PC2',
    hue='architecture_name', palette=architecture_color_mapping,
    data=pca_df, legend='full', s=100
)
plt.title('PCA on Dataset by Protein Architecture')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Architecture', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

scatter_plot_path = os.path.join(base_save_folder, 'pca_scatter_plot_by_architecture_with_kde.png')
plt.savefig(scatter_plot_path, bbox_inches='tight')
plt.close()

# ---------------------------------------------------------------------------------------------------


Variance explained by each component:
[0.34253849 0.22998887]


/var/folders/fg/0p2hf3h96gz7n16ykmw22n9w0000gn/T/ipykernel_894/1003490062.py:91: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(
/var/folders/fg/0p2hf3h96gz7n16ykmw22n9w0000gn/T/ipykernel_894/1003490062.py:91: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(


In [67]:
#ALL ARCHITECTURES WITH JUST ARCHETYPAL TOPOLOGIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
import seaborn as sns
import os
import json
import warnings


warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# ---------------------------------------------------------------------------------------------------
# Load the dataset

path = "/Volumes/dax-hd/project-data/search-files/merged-data.csv"
base_save_folder = "/Volumes/dax-hd/project-data/images/pca_architectures_topology_archetypes"
cath_dict_path = "/Volumes/dax-hd/project-data/search-files/cath-archetype-dict.txt"

df = pd.read_csv(path)

with open(cath_dict_path, 'r') as file:
    cath_dict = json.load(file)

if not os.path.exists(base_save_folder):
    os.makedirs(base_save_folder)

# ---------------------------------------------------------------------------------------------------

selected_columns = [
    # "composition_ALA","composition_CYS","composition_ASP","composition_GLU","composition_PHE","composition_GLY",
    # "composition_HIS","composition_ILE","composition_LYS","composition_LEU","composition_MET","composition_ASN",
    # "composition_PRO","composition_GLN","composition_ARG","composition_SER","composition_THR","composition_VAL",
    # "composition_TRP","composition_UNK","composition_TYR",
    # "ss_prop_alpha_helix","ss_prop_beta_bridge","ss_prop_beta_strand",
    # "ss_prop_3_10_helix","ss_prop_pi_helix",
    # "ss_prop_hbonded_turn","ss_prop_bend","ss_prop_loop",
    "hydrophobic_fitness",
    "isoelectric_point",
    "charge",
    "mass",
    "num_residues",
    "packing_density",
    "budeff_total",
    "evoef2_ref_total",
    "dfire2_total",
    "rosetta_total",
    "aggrescan3d_avg_value"
]
# ---------------------------------------------------------------------------------------------------

def filter_for_archetypes(df, cath_dict):
    archetype_ids = []
    for _, row in df.iterrows():
        class_num = str(row['Class number'])
        arch_num = str(row['Architecture number'])
        top_num = str(row['Topology number'])
        try:
            protein_id = cath_dict[class_num][arch_num][top_num]['protein_id']
            if protein_id[:4] in row['design_name']:
                archetype_ids.append(row['design_name'])
        except KeyError:
            continue
    return df[df['design_name'].isin(archetype_ids)]

df_filtered = filter_for_archetypes(df, cath_dict)

# ---------------------------------------------------------------------------------------------------

df_selected_cleaned = df_filtered[selected_columns].dropna()

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_selected_cleaned)

robust_scaler = RobustScaler()
scaled_features_robust = robust_scaler.fit_transform(df_selected_cleaned)

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features_robust)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

pca_df['architecture_name'] = df_filtered['architecture_name'].reset_index(drop=True)

print("Variance explained by each component:", pca.explained_variance_ratio_)

# Plotting PCA scatter with KDE
plt.figure(figsize=(30, 20))
unique_architectures = pca_df['architecture_name'].unique()
palette = sns.color_palette("tab20", n_colors=len(unique_architectures))
architecture_color_mapping = {arch: color for arch, color in zip(unique_architectures, palette)}

# KDE overlay for each architecture
for architecture, group_df in pca_df.groupby('architecture_name'):
    # Printing the number of data points per architecture
    print(f"Number of data points for architecture '{architecture}': {len(group_df)}")
    
    sns.kdeplot(
        x=group_df['PC1'], y=group_df['PC2'], 
        color=architecture_color_mapping[architecture],
        levels=2, fill=True, alpha=0.2
    )

# Scatter plot for visualization
sns.scatterplot(
    x='PC1', y='PC2',
    hue='architecture_name', palette=architecture_color_mapping,
    data=pca_df, legend='full', s=100
)
plt.title('PCA on Dataset by Protein Architecture')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Architecture', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

scatter_plot_path = os.path.join(base_save_folder, 'pca_scatter_plot_by_architecture_with_kde.png')
plt.savefig(scatter_plot_path, bbox_inches='tight')
plt.close()

# ---------------------------------------------------------------------------------------------------


Variance explained by each component: [0.61876376 0.11647977]
Number of data points for architecture 'aligned_prism (2,100)': 1
Number of data points for architecture 'alpha_alpha_barrel (1,50)': 6
Number of data points for architecture 'alpha_beta_barrel (3,20)': 41
Number of data points for architecture 'alpha_beta_complex (3,90)': 11
Number of data points for architecture 'alpha_beta_horseshoe (3,80)': 4
Number of data points for architecture 'alpha_beta_prism (3,65)': 2
Number of data points for architecture 'alpha_beta_roll (3,10)': 92
Number of data points for architecture 'alpha_horseshoe (1,25)': 19
Number of data points for architecture 'alpha_solenoid (1,40)': 2
Number of data points for architecture 'beta_barrel (2,40)': 118
Number of data points for architecture 'beta_complex (2,170)': 32
Number of data points for architecture 'beta_roll (2,30)': 68
Number of data points for architecture 'box (3,70)': 1
Number of data points for architecture 'clam (2,50)': 2
Number of data 

In [142]:
#ALL TOPOLOGIES BY ARCHITECTURES WITH JUST ARCHETYPES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
import seaborn as sns
import os
import json
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

# ---------------------------------------------------------------------------------------------------
# Load the dataset
path = "/Volumes/dax-hd/project-data/search-files/merged-data.csv"
base_save_folder = "/Volumes/dax-hd/project-data/images/pca_archetypal_"
cath_dict_path = "/Volumes/dax-hd/project-data/search-files/cath-archetype-dict.txt"

df = pd.read_csv(path)

with open(cath_dict_path, 'r') as file:
    cath_dict = json.load(file)
if not os.path.exists(base_save_folder):
    os.makedirs(base_save_folder)

# ---------------------------------------------------------------------------------------------------
# Add the architecture name to df
def add_topology_description(df, cath_dict):
    def get_topology_description(row):
        class_num = str(row['Class number'])
        arch_num = str(row['Architecture number'])
        top_num = str(row['Topology number'])
        try:
            description = cath_dict[class_num][arch_num][top_num]['description']
            return description
        except KeyError:
            return "Unknown"
    
    df['topology_description'] = df.apply(get_topology_description, axis=1)
    return df

df = add_topology_description(df, cath_dict)

# ---------------------------------------------------------------------------------------------------
# Filtering DF for archetypal topologies
def filter_for_archetypes(df_filtered, cath_dict):
    archetype_ids = []
    for _, row in df_filtered.iterrows():
        class_num = str(row['Class number'])
        arch_num = str(row['Architecture number'])
        top_num = str(row['Topology number'])
        try:
            protein_id = cath_dict[class_num][arch_num][top_num]['protein_id']
            if protein_id[:4] in row['design_name']:
                archetype_ids.append(row['design_name'])
        except KeyError:
            continue
    return df_filtered[df_filtered['design_name'].isin(archetype_ids)]

# ---------------------------------------------------------------------------------------------------

selected_architectures = ["sandwich (2,60)"]

selected_columns = [
    # "composition_ALA","composition_CYS","composition_ASP","composition_GLU","composition_PHE","composition_GLY",
    # "composition_HIS","composition_ILE","composition_LYS","composition_LEU","composition_MET","composition_ASN",
    # "composition_PRO","composition_GLN","composition_ARG","composition_SER","composition_THR","composition_VAL",
    # "composition_TRP","composition_UNK","composition_TYR",
    # "ss_prop_alpha_helix","ss_prop_beta_bridge","ss_prop_beta_strand","ss_prop_3_10_helix","ss_prop_pi_helix",
    # "ss_prop_hbonded_turn","ss_prop_bend","ss_prop_loop",
    "hydrophobic_fitness",
    "isoelectric_point",
    "charge",
    "mass",
    "num_residues",
    "packing_density",
    "budeff_total",
    "evoef2_ref_total",
    "dfire2_total",
    "rosetta_total",
    "aggrescan3d_avg_value"
]

# ---------------------------------------------------------------------------------------------------

for architecture_name in df['architecture_name'].unique():
    if architecture_name not in selected_architectures:
        continue
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_filtered = df[df['architecture_name'] == architecture_name]
    df_filtered_archetype = filter_for_archetypes(df_filtered, cath_dict)
    df_selected_cleaned = df_filtered_archetype[selected_columns].dropna()

    if df_selected_cleaned.empty or len(df_selected_cleaned) < 3:
        print(f"Skipping {architecture_name}: too few structures available.")
        continue
    else:
        print(f"pca complete for: {architecture_name}.")

    save_folder = os.path.join(base_save_folder, architecture_name.replace('/', '_'))
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # ---------------------------------------------------------------------------------------------------
    # Standardize and PCA
        
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_selected_cleaned)
    pca = PCA(n_components=0.95)
    principal_components = pca.fit_transform(scaled_features)
    if principal_components.shape[1] < 2:
        print(f"Warning: PCA for {architecture_name} resulted in fewer than 2 components.")
        pca_df = pd.DataFrame(data=principal_components, columns=['PC1'])
        pca_df['PC2'] = 0
    else:
        pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i}' for i in range(1, principal_components.shape[1] + 1)])
    
    pca_df['topology_description'] = df_filtered_archetype['topology_description'][df_selected_cleaned.index].reset_index(drop=True)

    # ---------------------------------------------------------------------------------------------------
    # Plotting

    plt.figure(figsize=(10, 7))
    unique_topologies = pca_df['topology_description'].unique()
    palette = sns.color_palette('tab20c', n_colors=len(unique_topologies))
    topology_color_mapping = {topology: color for topology, color in zip(unique_topologies, palette)}

    # KDE overlay for each architecture
    for topology, group_df in pca_df.groupby('topology_description'):
        sns.kdeplot(x=group_df['PC1'], y=group_df['PC2'], 
                    color=topology_color_mapping[topology],
                    levels=2, fill=True, alpha=0.3)

    scatter_plot = sns.scatterplot( x='PC1', y='PC2',
                                    hue='topology_description', palette=topology_color_mapping,
                                    data=pca_df, s=100)
    
    plt.title(f'PCA for {architecture_name}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    handles, labels = scatter_plot.get_legend_handles_labels()
    plt.legend([], [], frameon=False)
    plt.savefig(os.path.join(save_folder, 'pca_scatter_by_topology.png'), bbox_inches='tight')
    plt.close()

    # Save legend separately
    fig_legend = plt.figure(figsize=(3, 4))
    plt.figlegend(handles, labels, loc='center')
    plt.savefig(os.path.join(save_folder, 'legend.png'), bbox_inches='tight')
    plt.close(fig_legend)

    # Plotting cumulative variance
    plt.figure(figsize=(10, 7))
    plt.plot(range(1, pca.n_components_ + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--')
    plt.title('Cumulative Explained Variance')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.tight_layout()
    cumulative_variance_plot_path = os.path.join(save_folder, 'cumulative_explained_variance.png')
    plt.savefig(cumulative_variance_plot_path)
    plt.close()

    # Component loading plot
    for i in range(pca.n_components_):
        plt.figure(figsize=(10, 6))
        component_loadings = pca.components_[i]
        indices = np.argsort(abs(component_loadings))[::-1]
        feature_names = np.array(selected_columns)[indices]
        plt.bar(range(len(indices)), component_loadings[indices])
        plt.xticks(range(len(indices)), feature_names, rotation=90)
        plt.title(f'PCA Component {i+1} Loadings')
        plt.ylabel('Loading Value')
        feature_contribution_path = os.path.join(save_folder, f'pca_component_{i+1}_loadings.png')
        plt.tight_layout()
        plt.savefig(feature_contribution_path)
        plt.close()

pca complete for: sandwich (2,60).


In [160]:
#3D TOPOLOGIES BY ARCHITECTURES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import os
import json
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

# Load the dataset
path = "/Volumes/dax-hd/project-data/search-files/merged-data.csv"
base_save_folder = "/Volumes/dax-hd/project-data/images/3d_pca_topology"
cath_dict_path = "/Volumes/dax-hd/project-data/search-files/cath-archetype-dict.txt"

df = pd.read_csv(path)
with open(cath_dict_path, 'r') as file:
    cath_dict = json.load(file)

# Add the architecture name to df
def add_topology_description(df, cath_dict):
    def get_topology_description(row):
        class_num, arch_num, top_num = str(row['Class number']), str(row['Architecture number']), str(row['Topology number'])
        return cath_dict.get(class_num, {}).get(arch_num, {}).get(top_num, {}).get('description', "Unknown")
    
    df['topology_description'] = df.apply(get_topology_description, axis=1)
    return df

df = add_topology_description(df, cath_dict)

# Ensure the base_save_folder exists
if not os.path.exists(base_save_folder):
    os.makedirs(base_save_folder)

# Define the architectures to focus on and columns to be used
selected_architectures = ["sandwich (2,60)"]
selected_columns = [
    
    # "composition_ALA",
    # "composition_CYS","composition_ASP",
    # "composition_GLU",
    # "composition_PHE","composition_GLY",
    # "composition_HIS","composition_ILE","composition_LYS",
    # "composition_LEU",
    # "composition_MET","composition_ASN",
    # "composition_PRO","composition_GLN",
    # "composition_ARG","composition_SER","composition_THR",
    # "composition_VAL",
    # "composition_TRP","composition_UNK","composition_TYR",
    # "ss_prop_alpha_helix","ss_prop_beta_bridge","ss_prop_beta_strand","ss_prop_3_10_helix","ss_prop_pi_helix",
    # "ss_prop_hbonded_turn","ss_prop_bend","ss_prop_loop",
    # "hydrophobic_fitness",
    "isoelectric_point",
    # "charge",
    "mass",
    # "num_residues",
    # "packing_density",
    "budeff_total",
    # "evoef2_ref_total",
    # "dfire2_total",
    # "rosetta_total",
    "aggrescan3d_avg_value"
]

# Filter the DataFrame for the specific architectures
def filter_for_archetypes(df, cath_dict):
    archetype_ids = [row['design_name'] for _, row in df.iterrows() if cath_dict.get(str(row['Class number']), {}).get(str(row['Architecture number']), {}).get(str(row['Topology number']), {}).get('protein_id', '')[:4] in row['design_name']]
    return df[df['design_name'].isin(archetype_ids)]

df_filtered = filter_for_archetypes(df, cath_dict)

# Perform PCA and plot for each architecture
for architecture_name in df['architecture_name'].unique():
    if architecture_name not in selected_architectures:
        continue
    
    df_architecture = df[df['architecture_name'] == architecture_name]
    df_selected_cleaned = df_architecture[selected_columns].dropna()
    
    if df_selected_cleaned.empty:
        print(f"No data available for {architecture_name}. Skipping...")
        continue
    
    # Scaling
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_selected_cleaned)
    
    # Perform PCA for 3 components
    pca = PCA(n_components=3)
    principal_components = pca.fit_transform(scaled_features)
    pca_df = pd.DataFrame(principal_components, columns=['PC1', 'PC2', 'PC3'])
    pca_df['topology_description'] = df_architecture['topology_description'].loc[df_selected_cleaned.index].reset_index(drop=True)

    # Plotting
    fig = plt.figure(figsize=(30, 21))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(pca_df['PC1'], pca_df['PC2'], pca_df['PC3'], c=pd.factorize(pca_df['topology_description'])[0], cmap='tab20c', s=20)

    ax.set_title(f'3D PCA for {architecture_name}')
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    
    # Legend with unique labels
    unique_labels = pd.factorize(pca_df['topology_description'])[1]
    custom_legend = [plt.Line2D([0], [0], marker='o', linestyle='None', markersize=10, label=unique_labels[i], color=scatter.cmap(scatter.norm(i))) for i in range(len(unique_labels))]
    ax.legend(handles=custom_legend, bbox_to_anchor=(1.05, 1), loc='upper left', title="Topologies")
    
    plt.savefig(os.path.join(base_save_folder, f'{architecture_name.replace("/", "_")}_3d_pca.png'))
    plt.close()

print("Variance explained by each component:", (pca.explained_variance_ratio_))

Variance explained by each component: [0.53527937 0.24631892 0.21711078]
